In [1]:
import numpy as np
import pandas as pd
from datetime import date
import tensorflow as tf
import json
#import tensorflow_addons as tfa
import tensorflow.compat.v1 as tf1
from tensorflow import keras
from tensorflow.keras.layers import Activation
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.activations import *

# Sklearn imports
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

# Other imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Setting verbosity level of tensorflow training
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
#from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout,BatchNormalization, Embedding, LSTM, GRU
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# # Combine that shit
# # # import glob
# # os.chdir("./Kickstarter/")
# # extension = 'csv'
# # all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
# # #combine all files in the list
# # combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# # #export to csv
# # combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

# #Remove Duplicates
# # df = pd.read_csv('./Cleaned_data')
# # df = df.drop_duplicates(subset=['id'])
# # df.to_csv("Cleaned_data.csv", index=False, encoding='utf-8-sig')

# # New Files
# df = pd.read_csv('./Cleaned_data.csv')
# df = df.drop(columns=['backers_count', 'country', 'country_displayable_name', 'created_at', 'creator', 'currency_symbol', 'currency_trailing_code', 'current_currency',
#                       'disable_communication', 'friends', 'fx_rate', 'id', 'is_backing', 'is_starrable', 'is_starred', 'permissions', 'photo', 'slug', 'source_url', 'spotlight',
#                       'staff_pick', 'state_changed_at', 'static_usd_rate', 'urls', 'usd_type', 'usd_exchange_rate', 'usd_pledged', 'profile', 'pledged', 'location', 'currency',
#                       'converted_pledged_amount'], axis=1)

# df['launch_to_deadline_days'] = round((df['deadline'] - df['launched_at'])/86400, 0)
# df = df.drop(columns=['deadline', 'launched_at'], axis=1)

# df['blurb_len'] = df['blurb'].map(str).apply(len)
# df = df.drop(columns=['blurb'], axis=1)

# df['name_len'] = df['name'].map(str).apply(len)
# df = df.drop(columns=['name'], axis=1)

# df['temp_cat'] = df['category'].apply(json.loads)
# df2 = df.temp_cat.apply(pd.Series)
# df['category'] = df2['parent_name']
# df = df.drop(columns='temp_cat', axis=1)
# df = df.drop(df[df['state'] == 'canceled'].index)
# df = df.drop(df[df['state'] == 'live'].index)
# df = df[['name_len', 'blurb_len', 'goal', 'launch_to_deadline_days', 'category', 'state']]
# df.loc[df['state'] == 'successful', 'SuccessfulBool'] = 1
# df.loc[df['state'] == 'failed', 'SuccessfulBool'] = 0
# df = df.drop(columns='state')

# # Old Files
# df1 = pd.read_csv('./kickstarter_data_full.csv')
# df1 = df1.drop(df1[df1['state'] == 'canceled'].index)
# df1 = df1.drop(df1[df1['state'] == 'live'].index)
# df1 = df1.drop(df1[df1['state'] == 'suspended'].index)
# df1 = df1.drop(df1[df1['TOPCOUNTRY'] == 0].index)
# df1 = df1[['name_len', 'blurb_len', 'goal', 'launch_to_deadline_days', 'category', 'SuccessfulBool']]

# # Concat
# df = pd.concat([df, df1])

# # Data/Target prep
y = df.SuccessfulBool
X = df[['name_len', 'blurb_len', 'goal', 'launch_to_deadline_days', 'category']]
# X['category'] = df['category'].astype('category')
# X['category'] = X['category'].cat.codes
X = pd.get_dummies(X, columns=['category'], drop_first=True)

In [3]:
# from tensorflow.keras import backend as k
# class CustomCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs=None):
#         if (logs.get('val_acc') + 5) < logs.get('acc')  :
#             self.model.stop_training = True
#         # gc.collect()
#         # k.clear_session()
# my_callback_object = CustomCallback()

In [4]:
METRICS = [
  # keras.metrics.TruePositives(name='tp'),
  # keras.metrics.FalsePositives(name='fp'),
  # keras.metrics.TrueNegatives(name='tn'),
  # keras.metrics.FalseNegatives(name='fn'), 
  keras.metrics.Precision(name='precision'),
  keras.metrics.Recall(name='recall'),
  #keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
  keras.metrics.BinaryAccuracy(name='acc'),
  keras.metrics.AUC(name='auc'),
]

In [5]:
from numpy import save
from numpy import load
# X = np.array(X)
# y = np.array(y)

# Save data
# save('X.npy', X)
# save('y.npy', y)

# Load data
X = load('X.npy')
y = load('y.npy')

In [6]:
lengthx = 256
kern_init='he_uniform'
bias_init='he_uniform'
input_dims=41

def identify_block(input_data):
    x_shortcut = input_data
    
    #First Component
    x = layers.Dense(lengthx, kernel_initializer=kern_init, bias_initializer=bias_init)(input_data)#, activation=activation)(input_data)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    
    #Second Component
    x = layers.Dense(lengthx, kernel_initializer=kern_init, bias_initializer=bias_init)(x)#, activation=activation)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    
    #Third Component
    x = layers.Dense(lengthx, kernel_initializer=kern_init, bias_initializer=bias_init)(x)#, activation=activation)(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    
    #Final
    x = layers.Add()([x, x_shortcut])
    x = layers.Activation(tf.keras.activations.relu)(x)
    return x

def conv_block(input_data):
    x_shortcut = input_data
    
    #First Component
    x = layers.Dense(lengthx, kernel_initializer=kern_init, bias_initializer=bias_init)(input_data)#, activation=activation)(input_data)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    #Second Component
    x = layers.Dense(lengthx, kernel_initializer=kern_init, bias_initializer=bias_init)(input_data)#, activation=activation)(input_data)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    #Third Component
    x = layers.Dense(lengthx, kernel_initializer=kern_init, bias_initializer=bias_init)(x)#, activation=activation)(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    #Shortcut path
    x_shortcut = layers.Dense(lengthx, kernel_initializer=kern_init, bias_initializer=bias_init)(x_shortcut)
    x_shortcut = layers.BatchNormalization()(x_shortcut)
    #Final
    x = layers.Add()([x, x_shortcut])
    x = layers.Activation(tf.keras.activations.relu)(x)
    return x

def KSmodel(optimizer='adam', kernel_initializer='he_uniform', dropout=0, activation='elu'):
        
    inputs = keras.Input(shape=input_dims)
    x = layers.Dense(lengthx, kernel_initializer=kern_init, bias_initializer=bias_init)(inputs)#, activation=activation)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    
    # Stage 2
    x = conv_block(x)
    x = identify_block(x)
    x = identify_block(x)
            
    # Stage 3
    x = conv_block(x)
    x = identify_block(x)
    x = identify_block(x)
    x = identify_block(x)
    
    # Stage 4
    x = conv_block(x)
    x = identify_block(x)
    x = identify_block(x)
    x = identify_block(x)
    x = identify_block(x)
    
    # Stage 5
    x = conv_block(x)
    x = identify_block(x)
    x = identify_block(x)
    x = identify_block(x)
    x = identify_block(x)
    
    # Stage 6
    x = conv_block(x)
    x = identify_block(x)
    x = identify_block(x)
    x = identify_block(x)
    x = identify_block(x)
        
    # Stage 7
    x = conv_block(x)
    x = identify_block(x)
    x = identify_block(x)
    x = identify_block(x)
    x = identify_block(x)
    
    # Stage 8
    x = conv_block(x)
    x = identify_block(x)
    x = identify_block(x)
    x = identify_block(x)
    x = identify_block(x)
    
    # Stage 9
    x = conv_block(x)
    x = identify_block(x)
    x = identify_block(x)
    x = identify_block(x)
    x = identify_block(x)
        
    outputs = layers.Dense(1, kernel_initializer=kern_init, bias_initializer=bias_init, activation='sigmoid')(x)
       
    model = keras.Model(inputs, outputs)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)

    return model

def SimpleModel(optimizer='adam', kernel_initializer='he_uniform', dropout=0, activation='relu'):
    model = Sequential()
    model.add(Dense(256, activation=activation))
    model.add(BatchNormalization())
    model.add(Dense(64, activation=activation))
    model.add(BatchNormalization())
    model.add(Embedding(input_dim=input_dims, output_dim=64))
    model.add(GRU(16, return_sequences=True))
    model.add(BatchNormalization())
    # model.add(GRU(16, return_sequences=True))
    # model.add(BatchNormalization())
    # model.add(GRU(16, return_sequences=True))
    # model.add(BatchNormalization())
    # model.add(GRU(16, return_sequences=True))
    # model.add(BatchNormalization())
    # model.add(GRU(16, return_sequences=True))
    model.add(GRU(4, return_sequences=False))
    
    # model.add(Dense(1024, activation=activation))
    # model.add(BatchNormalization())
    # model.add(Dense(1024, activation=activation))
    # model.add(BatchNormalization())
    # model.add(Dense(32, activation=activation))
    # model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)
    return model

clf = KerasClassifier(build_fn=KSmodel, verbose=1, epochs=2000, batch_size=5120, validation_split=0.1)#, callbacks=[my_callback_object])#, validation_split=0.1)#data=(X_val, y_val))

In [7]:
model_dt = clf
#model_dt = make_pipeline(OrdinalEncoder(), SimpleImputer(strategy='median'), QuantileTransformer(output_distribution='uniform'), clf)
#model_dt = make_pipeline(OrdinalEncoder(), clf)#SimpleImputer(strategy='mean'), clf)
model_dt.fit(X, y)

Train on 186145 samples, validate on 20683 samples
Epoch 1/2000
186145/186145 [==============================] - 28s 151us/sample - loss: 0.7478 - precision: 0.6353 - recall: 0.8604 - acc: 0.6199 - auc: 0.5714 - val_loss: 382.2457 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_acc: 0.5673 - val_auc: 0.5000
Epoch 2/2000
186145/186145 [==============================] - 6s 33us/sample - loss: 0.6509 - precision: 0.6423 - recall: 0.8939 - acc: 0.6377 - auc: 0.6168 - val_loss: 1.1257 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_acc: 0.5673 - val_auc: 0.6538
Epoch 3/2000
186145/186145 [==============================] - 6s 33us/sample - loss: 0.6474 - precision: 0.6425 - recall: 0.8977 - acc: 0.6390 - auc: 0.6278 - val_loss: 0.7535 - val_precision: 0.4371 - val_recall: 0.9974 - val_acc: 0.4431 - val_auc: 0.5787
Epoch 4/2000
186145/186145 [==============================] - 6s 33us/sample - loss: 0.6465 - precision: 0.6450 - recall: 0.8891 - acc: 0.6399 - auc: 0.6300 -